In [14]:
import pandas as pd
import numpy as np
import os

raw_data_dir = '../../../data/raw/usa'
int_data_dir = '../../../data/interim/usa'
proc_data_dir = '../../../data/processed/adm1'

In [15]:
print("{y:04d}".format(y=1))

0001


In [21]:
from importlib import reload
import merge_policy_and_cases
reload(merge_policy_and_cases)

df_rows_merged, policy_data_keys = merge_policy_and_cases.download_and_process_policy_csv()

In [22]:
df_rows_merged

,date,adm0_name,adm1_name,school_closure,school_closure_popweighted,school_closure_optional,school_closure_optional_popweighted,business_closure,business_closure_popweighted,business_closure_optional,...,home_isolation_optional_popweighted,paid_sick_leave,paid_sick_leave_popweighted,paid_sick_leave_optional,paid_sick_leave_optional_popweighted,emergency_declaration,emergency_declaration_popweighted,date_to_sort,travel_countries,no_gathering_size
0,2020-01-31,USA,all,0,0.000000,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,1,1.0,2020-01-31,[nan],NaN
1,2020-02-01,USA,New York,0,0.000000,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,2020-02-01,[nan],NaN
2,2020-02-02,USA,all,0,0.000000,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,2020-02-02,"[China, all]",NaN
3,2020-02-19,USA,all,0,0.000000,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,2020-02-19,"Hong Kong, Japan",NaN
4,2020-02-22,USA,all,0,0.000000,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,2020-02-22,"[South korea, japan]",NaN
5,2020-02-23,USA,all,0,0.000000,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,2020-02-23,"Iran, Ialy",NaN
6,2020-02-24,USA,all,0,0.000000,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,2020-02-24,"[Iran, Italy, South korea]",NaN
7,2020-02-27,USA,Washington,1,0.005654,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,2020-02-27,[nan],NaN
8,2020-02-28,USA,New York,0,0.000000,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,2020-02-28,[nan],NaN
9,2020-02-29,USA,Washington,0,0.000000,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,1,1.0,2020-02-29,"[nan, nan, nan]",NaN


# 4. Merge policy data with cases data

In [23]:
cases_data = pd.read_csv(os.path.join(int_data_dir,"usa_jhu_cases_state.csv"))

policy_data_by_state = df_rows_merged.groupby('adm1_name')

In [24]:
# testing regime data
testing_regimes = pd.read_csv(os.path.join(int_data_dir,'usa_states_covidtrackingdotcom_int_with_testing_regimes.csv'))

# add datetime to cases data for easy sorting
cases_data['date_to_sort'] = pd.to_datetime(cases_data['date'])
cases_data = cases_data.sort_values(['date_to_sort','adm1_name'])#.reset_index()

num_cases_data = len(cases_data)
# add all the columns we're gonna need
policy_keys_popweighted = [x+"_popweighted" for x in policy_keys]

# add empty olumns
for policy_key in policy_keys:
    cases_data[policy_key] = np.zeros(num_cases_data)
    
for policy_key_popweighted in policy_keys_popweighted:
    cases_data[policy_key_popweighted] = np.zeros(num_cases_data)

cases_data['travel_countries'] = [[] for i in range(num_cases_data)]


    

In [25]:
policies_affecting_all_states[policies_affecting_all_states['travel_ban_international_outbound']>0]

NameError: name 'policies_affecting_all_states' is not defined

In [243]:
# now do the state specific ones
cases_data_by_state = cases_data.groupby('adm1_name')
cases_data_by_state

In [244]:
state_names_with_polices = list(policy_data_by_state.groups.keys())
if "all" in list(cases_data_by_state.groups.keys()):
    state_names_with_polices.remove('all')

# for each state
for state in state_names_with_polices:
    if not state in cases_data_by_state.groups.keys():
        continue
        
    cases_this_state = cases_data_by_state.get_group(state).reset_index()

    policies_this_state = policy_data_by_state.get_group(state).reset_index(drop=True)
    
    # for each state policy
    for i in range(len(policies_this_state)):
        this_policy = policies_this_state.loc[i]
    
        date_this_policy = this_policy['date_to_sort']
    
        # get idxs back into the original df
        affected_case_rows = cases_this_state[cases_this_state['date_to_sort']>= date_this_policy]['index'].values
        for policy_key in policy_keys:
            cases_data.loc[affected_case_rows,policy_key] = np.maximum(cases_data.loc[affected_case_rows,policy_key].values,
                                                                   this_policy[policy_key]) 
            
        for policy_key in policy_keys_popweighted:
            frac_summed = cases_data.loc[affected_case_rows,policy_key].values + this_policy[policy_key]
            cases_data.loc[affected_case_rows,policy_key] = np.minimum( frac_summed, 1.0)
            
            
            

In [313]:
# 5. Merge in with covidtracking.com data for testing_regime
# add a nan column to testing_regime
cases_data['testing_regime'] = float("NaN")

testing_regimes = pd.read_csv(os.path.join(int_data_dir,'usa_states_covidtrackingdotcom_int_with_testing_regimes.csv'))


In [333]:
testing_regimes = testing_regimes[['date', 'adm1_name', 'testing_regime']].sort_values('date')
testing_regimes['adm1_name'] = testing_regimes['adm1_name']#.apply(acc_to_statename)
testing_regimes_by_state = testing_regimes.groupby('adm1_name')

In [345]:
for state in cases_data_by_state.groups.keys():
    cases_this_state = cases_data_by_state.get_group(state).reset_index()
    testing_regimes_this_state_all = testing_regimes_by_state.get_group(state)[['date','testing_regime']]
    testing_regimes_this_state = pd.DataFrame(cases_this_state['date']).merge(testing_regimes_this_state_all,
                                                                           how='left')

    # first backfill anything
    testing_regimes_this_state = testing_regimes_this_state.fillna(method='ffill')
    # now fill any nans with 0
    testing_regimes_this_state = testing_regimes_this_state.fillna(0)
    
    rows_this_state = cases_this_state['index'].values

    # now add to cases_data
    cases_data.loc[rows_this_state,'testing_regime'] = testing_regimes_this_state['testing_regime'].values

In [346]:
# publish
cases_data_to_publish = cases_data.drop(['adm2_name','date_to_sort'],axis=1).reset_index(drop=True)
cases_data_to_publish.to_csv(os.path.join(proc_data_dir,'USA_processed.csv'),index=False)